In [3]:

from skyfield.api import load
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', 10) 
pd.set_option('display.max_rows', 10000)
pd.set_option('display.width', 300)
pd.set_option('display.max_colwidth', 300)

eph = load('de422.bsp')

planets = {
        'sun': eph['sun'],
        'mercury': eph['mercury'],
        'venus': eph['venus'],
        'earth': eph['earth'],
        'moon': eph['moon'],
        'mars': eph['mars'],
        'jupiter': eph['jupiter barycenter'],
        'saturn': eph['saturn barycenter'],
        'uranus': eph['uranus barycenter'],
        'neptune': eph['neptune barycenter'],
        'pluto': eph['pluto barycenter'] }

earth = planets['earth']

In [4]:
def in_conjunction(angles):
    sorted_angles = np.sort(angles.values)
    # sorted_angles_shifted = sorted_angles.take(range(1, len(angles)+1), mode='wrap')
    sorted_angles_shifted = np.roll(sorted_angles, -1)
    # print("sorted_angles = ", sorted_angles, "sorted_angles_shifted = ", sorted_angles_shifted)
    sorted_angles_diff = sorted_angles_shifted - sorted_angles
    sorted_angles_diff[-1] += 360.0
    sorted_angles_diff = list(map(lambda x: 360.0 - x if (x > 180.0) else x, sorted_angles_diff))
    # print(sorted_angles_diff)
    max_angle = max(sorted_angles_diff)
    return True if (max_angle < conjunction_span_degrees) else False

In [5]:
planet_names_of_interest = ['sun', 'mercury', 'venus', 'mars', 'jupiter', 'saturn']
conjunction_span_degrees = 45

In [6]:
ts = load.timescale(builtin=True)
t = ts.utc(1600, 1, range(0, 500*366, 1), 0, 0, 0)

In [9]:
t

<Time tt=[2305446.5004882407 ... 2488445.500800741] len=183000>

In [ ]:
df = pd.DataFrame(index=t.utc_datetime(), columns=planet_names_of_interest)

In [10]:
df

,sun,mercury,venus,mars,jupiter,saturn
1599-12-31 00:00:00+00:00,284.549682,291.395966,263.792748,143.216989,146.549831,212.905649
1600-01-01 00:00:00+00:00,285.568778,293.043846,263.540332,143.061517,146.476452,212.967361
1600-01-02 00:00:00+00:00,286.587841,294.695831,263.329079,142.892569,146.400246,213.027604
1600-01-03 00:00:00+00:00,287.606878,296.351381,263.159480,142.710204,146.321252,213.086362
1600-01-04 00:00:00+00:00,288.625892,298.009793,263.031763,142.514510,146.239514,213.143622
...,...,...,...,...,...,...
2101-01-08 00:00:00+00:00,286.091197,305.336596,273.203091,257.591192,226.739219,216.031741
2101-01-09 00:00:00+00:00,287.110187,306.209886,274.459277,258.306913,226.895368,216.095795
2101-01-10 00:00:00+00:00,288.129065,306.960708,275.715440,259.023249,227.049679,216.158398
2101-01-11 00:00:00+00:00,289.147828,307.574046,276.971579,259.740203,227.202129,216.219537


In [7]:
for pn in planet_names_of_interest: 
    print("Computing coordinates of {}".format(pn))
    observer = earth.at(t)
    planet = planets[pn]
    lat, lon, distance = observer.observe(planet).ecliptic_latlon()
    df.loc[:, pn] = lon.degrees

Computing coordinates of sun
Computing coordinates of mercury
Computing coordinates of venus
Computing coordinates of mars
Computing coordinates of jupiter
Computing coordinates of saturn


In [8]:
df

,sun,mercury,venus,mars,jupiter,saturn
1599-12-31 00:00:00+00:00,284.549682,291.395966,263.792748,143.216989,146.549831,212.905649
1600-01-01 00:00:00+00:00,285.568778,293.043846,263.540332,143.061517,146.476452,212.967361
1600-01-02 00:00:00+00:00,286.587841,294.695831,263.329079,142.892569,146.400246,213.027604
1600-01-03 00:00:00+00:00,287.606878,296.351381,263.159480,142.710204,146.321252,213.086362
1600-01-04 00:00:00+00:00,288.625892,298.009793,263.031763,142.514510,146.239514,213.143622
...,...,...,...,...,...,...
2101-01-08 00:00:00+00:00,286.091197,305.336596,273.203091,257.591192,226.739219,216.031741
2101-01-09 00:00:00+00:00,287.110187,306.209886,274.459277,258.306913,226.895368,216.095795
2101-01-10 00:00:00+00:00,288.129065,306.960708,275.715440,259.023249,227.049679,216.158398
2101-01-11 00:00:00+00:00,289.147828,307.574046,276.971579,259.740203,227.202129,216.219537


In [11]:
print("Calculating conjunctions...")

for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    result = in_conjunction(row)
    df.at[index, 'in_conjunction'] = result

in_conjunction = df.loc[:, 'in_conjunction']
in_conjunction_shifted = np.roll(in_conjunction, 1)
df.loc[:, 'conjunction_change'] = in_conjunction ^ in_conjunction_shifted;
conjunction_change = df.loc[:, 'conjunction_change']

df.loc[:, 'conjunction_start'] = in_conjunction & conjunction_change;
df.loc[:, 'conjunction_end'] = np.invert(in_conjunction) & conjunction_change;

  0%|          | 0/183000 [00:00<?, ?it/s]

Calculating conjunctions...


100%|██████████| 183000/183000 [01:11<00:00, 2559.40it/s]


In [12]:
print("Post processing conjunction data ...")

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if (row['conjunction_start']):
        df.at[index, 'label'] = 'Conjunction Start'
    elif (row['conjunction_end']):
        df.at[index, 'label'] = 'Conjunction End'

print(df.loc[df['conjunction_change'] == True, planet_names_of_interest + ['label']])


  0%|          | 0/183000 [00:00<?, ?it/s]

Post processing conjunction data ...


100%|██████████| 183000/183000 [01:12<00:00, 2537.53it/s]


                                  sun     mercury       venus        mars     jupiter      saturn              label
1602-10-17 00:00:00+00:00  208.919908  208.105226  248.433165  252.990376  216.144312  236.577336  Conjunction Start
1602-10-31 00:00:00+00:00  222.916395  230.710730  265.001227  263.333751  219.211244  238.157415    Conjunction End
1624-07-27 00:00:00+00:00  129.390750  125.714842  113.873057  136.525042  158.352808  145.177166  Conjunction Start
1624-09-23 00:00:00+00:00  185.539461  198.168163  185.658043  173.489494  170.702955  152.375835    Conjunction End
1626-09-08 00:00:00+00:00  170.359384  194.894747  203.002381  175.502347  214.746049  173.646540  Conjunction Start
1626-09-23 00:00:00+00:00  185.031508  210.444193  221.055000  185.222611  217.634078  175.515596    Conjunction End
1662-11-06 00:00:00+00:00  228.562126  247.582552  205.224841  244.077397  236.978071  249.579826  Conjunction Start
1663-01-12 00:00:00+00:00  296.624275  278.203719  289.289639  2